### Mobility processing
This notebook works through the process of adapting Google mobility data into the required format for the model.

In [ ]:
import pandas as pd

from emutools.tex import StandardTexDoc, remove_underscore_multiindexcol
from emutools.calibration import round_sigfig
from aust_covid.inputs import get_raw_state_mobility
from emutools.utils import add_image_to_doc
from aust_covid.mobility import get_non_wa_mob_averages, get_relative_mobility, map_mobility_locations
from aust_covid.plotting import plot_state_mobility, plot_processed_mobility
from inputs.constants import MOBILITY_MAP, SUPPLEMENT_PATH, MOBILITY_AVERAGE_WINDOW

In [ ]:
app_doc = StandardTexDoc(SUPPLEMENT_PATH, 'mobility', 'Mobility processing', 'austcovid')

In [ ]:
state_data, jurisdictions, mob_locs = get_raw_state_mobility(app_doc)
fig = plot_state_mobility(state_data, jurisdictions, mob_locs)
fig

In [ ]:
caption = 'Locations: Retail and recreation (green), grocery and pharmacy (purple), parks (orange), transit stations (yellow), workplaces (blue), and residential (red).'
add_image_to_doc(fig.update_layout(showlegend=False), 'state_mobility', 'svg', 'Raw state-level mobility obtained from Google.', app_doc, 'Mobility extension', caption=caption)

In [ ]:
mapping_table = pd.DataFrame(MOBILITY_MAP)
mapping_table.index = mapping_table.index.str.replace('_', ' ').str.capitalize()
mapping_table.columns = mapping_table.columns.str.replace('_', ' ').str.capitalize()
mapping_table = mapping_table.applymap(str)
app_doc.include_table(mapping_table, 'mob_map', 'Mapping of mobility to mixing matrix locations.', 'Mobility extension')
mapping_table

In [ ]:
wa_data = state_data.loc[state_data['sub_region_1'] == 'Western Australia', mob_locs]
state_averages = get_non_wa_mob_averages(state_data, mob_locs, jurisdictions, app_doc)
non_wa_relmob = get_relative_mobility(state_averages)
wa_relmob = get_relative_mobility(wa_data)
processed_mob = map_mobility_locations(wa_relmob, non_wa_relmob, app_doc)
smoothed_mob = processed_mob.rolling(MOBILITY_AVERAGE_WINDOW).mean().dropna()
squared_mob = smoothed_mob ** 2.0

In [ ]:
mobility_types = {
    'original': remove_underscore_multiindexcol(processed_mob),
    'smoothed': remove_underscore_multiindexcol(smoothed_mob),
    'smoothed squared': remove_underscore_multiindexcol(squared_mob),
}
fig = plot_processed_mobility(mobility_types)
fig

In [ ]:
caption = 'Work mobility (yellow line), smoothed work mobility (blue line), squared smoothed work mobility (red line), ' \
    'averaged other locations mobility (green line), smoothed averaged other locations mobility (purple), and squared smoothed averaged other locations mobility (orange).'
add_image_to_doc(fig.update_layout(showlegend=False), 'processed_mobility', 'svg', 'Processed mobility for model.', app_doc, 'Mobility extension', caption=caption)

In [ ]:
app_doc.write_doc()